In [4]:
import itertools
import os
import time
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import Accelerator
import numpy as np
import matplotlib.pyplot as plt

In [5]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("lmsys/lmsys-chat-1m")

In [6]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name)

model.eval()
device = 'cuda:1'
model.to(device)

if not tokenizer.chat_template:
    tokenizer.chat_template = """
    {system_message}
    {user_message}
    {assistant_message}
    """

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.99s/it]


In [20]:
c = 0

for conversation in itertools.islice(ds['train'], 100):
    turns = int(conversation['turn'])
    print(f"Conversation {c}")
    for i in range(turns):
        # Prepare messages for the current turn
        messages = [conversation['conversation'][j] for j in range(2*i+1)]

        # Tokenize the conversation
        tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, return_tensors="pt")
        tokenized_chat = tokenized_chat.to(device)
        
        input_tokens = tokenized_chat.size(1)
        print(f'Input token count: {input_tokens}')

        out_tokens = tokenizer.apply_chat_template([conversation['conversation'][2*i+1]], tokenize=True, return_tensors="pt").size(1)
        print(f'Output token count: {out_tokens}')

        # Generate a response
        start = time.time()
        outputs = model.generate(input_ids=tokenized_chat,
                                 max_new_tokens=out_tokens,
                                 num_beams=1,
                                 early_stopping=False)
        latency = time.time() - start

        # Decode the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f'Input: {input_tokens}, Output: {out_tokens}, Latency: {latency} seconds')
    c+=1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Conversation 0
Input token count: 13
Output token count: 234


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Input: 13, Output: 234, Latency: 9.540157556533813 seconds
Conversation 1
Input token count: 47
Output token count: 364


KeyboardInterrupt: 